<a href="https://colab.research.google.com/github/dangduong2806/NMT-project/blob/main/Hu%E1%BA%A5n%20luy%E1%BB%87n%20transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nếu đã tải các file về local

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving vietnam_validate.txt to vietnam_validate.txt


Load tokenizer cho mỗi language

In [ ]:
from tokenizers import Tokenizer

In [ ]:
source_tokenizer = Tokenizer.from_file('/content/300kenglish_bpe_bytelevel.json')
target_tokenizer = Tokenizer.from_file('/content/300kvietnamese_bpe_bytelevel.json')

Tạo list các id của tập training

In [ ]:
src_ids = []
with open("100kenglish.txt", "r", encoding="utf-8") as f:
  for line in f:
    line = line.strip()
    if line:
      output = source_tokenizer.encode(line)
      id = output.ids
      sos_id = source_tokenizer.token_to_id("<sos>")
      eos_id = source_tokenizer.token_to_id("<eos>")
      id = [sos_id] + id + [eos_id]
      src_ids.append(id)

In [ ]:
tgt_ids = []
with open("100kvietnam.txt", "r", encoding="utf-8") as f:
  for line in f:
    line = line.strip()
    if line:
      output = target_tokenizer.encode(line)
      id = output.ids
      sos_id = target_tokenizer.token_to_id("<sos>")
      eos_id = target_tokenizer.token_to_id("<eos>")
      id = [sos_id] + id + [eos_id]
      tgt_ids.append(id)

Tạo list các id của tập validate

In [ ]:
src_ids_validate = []
with open("english_validate.txt", "r", encoding="utf-8") as f:
  for line in f:
    line = line.strip()
    if line:
      output = source_tokenizer.encode(line)
      id = output.ids
      sos_id = source_tokenizer.token_to_id("<sos>")
      eos_id = source_tokenizer.token_to_id("<eos>")
      id = [sos_id] + id + [eos_id]
      src_ids_validate.append(id)

In [ ]:
tgt_ids_validate = []
with open("vietnam_validate.txt", "r", encoding="utf-8") as f:
  for line in f:
    line = line.strip()
    if line:
      output = target_tokenizer.encode(line)
      id = output.ids
      sos_id = target_tokenizer.token_to_id("<sos>")
      eos_id = target_tokenizer.token_to_id("<eos>")
      id = [sos_id] + id + [eos_id]
      tgt_ids_validate.append(id)

Padding các vector id về cùng một kích thước

In [ ]:
import torch
from torch.utils.data import Dataset

class TranslationDataset(Dataset):
    def __init__(self, src_data, tgt_data):
        self.src = src_data
        self.tgt = tgt_data

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        return {
            'src': torch.tensor(self.src[idx], dtype=torch.long),
            'tgt': torch.tensor(self.tgt[idx], dtype=torch.long)
        }


In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    src_pad_id = source_tokenizer.token_to_id("<pad>")
    tgt_pad_id = target_tokenizer.token_to_id("<pad>")

    src_batch = [item['src'] for item in batch]
    tgt_batch = [item['tgt'] for item in batch]

    src_pad = pad_sequence(src_batch, batch_first=True, padding_value=src_pad_id)
    tgt_pad = pad_sequence(tgt_batch, batch_first=True, padding_value=tgt_pad_id)

    return {
        'src': src_pad,
        'tgt': tgt_pad
    }

Chỉnh sửa batch size

In [ ]:
from torch.utils.data import DataLoader

dataset = TranslationDataset(src_ids, tgt_ids)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=collate_fn, num_workers=0)

In [ ]:
validate = TranslationDataset(src_ids_validate, tgt_ids_validate)
valid_data = DataLoader(validate, batch_size=64, shuffle=True, collate_fn=collate_fn, num_workers=0)

Kiểm tra thử dataloader và valid_data

In [ ]:
for batch in dataloader:
    print("SRC batch:")
    print(batch['src'])
    print("SRC shape:", batch['src'].shape)

    print("\nTGT batch:")
    print(batch['tgt'])
    print("TGT shape:", batch['tgt'].shape)

    break  # chỉ in 1 batch đầu tiên

SRC batch:
tensor([[  2, 123, 435,  ...,   0,   0,   0],
        [  2, 243, 160,  ...,   0,   0,   0],
        [  2, 140, 125,  ...,   0,   0,   0],
        ...,
        [  2, 243, 284,  ...,   0,   0,   0],
        [  2, 171, 155,  ...,   0,   0,   0],
        [  2, 123, 541,  ...,   0,   0,   0]])
SRC shape: torch.Size([64, 83])

TGT batch:
tensor([[  2, 874, 398,  ...,   0,   0,   0],
        [  2, 481, 669,  ...,   0,   0,   0],
        [  2, 162, 205,  ...,   0,   0,   0],
        ...,
        [  2, 445, 765,  ...,   0,   0,   0],
        [  2, 365, 254,  ...,   0,   0,   0],
        [  2, 169, 246,  ...,   0,   0,   0]])
TGT shape: torch.Size([64, 90])


In [ ]:
for batch in valid_data:
    print("SRC batch:")
    print(batch['src'])
    print("SRC shape:", batch['src'].shape)

    print("\nTGT batch:")
    print(batch['tgt'])
    print("TGT shape:", batch['tgt'].shape)

    break  # chỉ in 1 batch đầu tiên

SRC batch:
tensor([[   2,  165, 1062,  ...,    0,    0,    0],
        [   2,  189,  155,  ...,   11,    3,    0],
        [   2,  151, 3660,  ...,    0,    0,    0],
        ...,
        [   2,  221,  398,  ...,    0,    0,    0],
        [   2,  123,    6,  ...,    0,    0,    0],
        [   2,  140,  189,  ...,    0,    0,    0]])
SRC shape: torch.Size([64, 74])

TGT batch:
tensor([[   2,  432,  246,  ...,    0,    0,    0],
        [   2,  395,  245,  ...,    0,    0,    0],
        [   2,  265, 2509,  ...,    0,    0,    0],
        ...,
        [   2,  301,  182,  ...,    0,    0,    0],
        [   2,  169,  246,  ...,    0,    0,    0],
        [   2,  162,  319,  ...,    0,    0,    0]])
TGT shape: torch.Size([64, 114])


Cài đặt mô hình

In [ ]:
import torch
import torch.nn as nn

class BaseSeq2Seq(nn.Module):
    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        """Train-time forward pass with teacher forcing"""
        raise NotImplementedError

    def generate(self, src, max_len=100):
        """Inference-time decoding"""
        raise NotImplementedError

In [ ]:
vocab_src = source_tokenizer.get_vocab()
vocab_tgt = target_tokenizer.get_vocab()

In [ ]:
print("Source tokenizer vocab size:", source_tokenizer.get_vocab_size())
print("Target tokenizer vocab size:", target_tokenizer.get_vocab_size())

Source tokenizer vocab size: 16000
Target tokenizer vocab size: 16000


Hàm huấn luyện và validate

In [ ]:
from tqdm import tqdm
import wandb

In [ ]:
# train.py
import torch
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0

    print(">> Start training")
    loop = tqdm(dataloader, total=len(dataloader), desc="training", leave=True, colour="green")
    for batch in loop:
        src = batch['src'].to(device)
        tgt = batch['tgt'].to(device)

        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]

        optimizer.zero_grad()
        output = model(src, tgt_input)

        output = output.reshape(-1, output.shape[-1])
        tgt_output = tgt_output.reshape(-1)

        loss = criterion(output, tgt_output)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # Cập nhật thanh progress bar
        loop.set_postfix(loss=loss.item())
        wandb.log({"batch_loss": loss.item()})

    return total_loss / len(dataloader)

def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
      loop = tqdm(dataloader, total=len(dataloader), desc="validating", leave=True, colour="blue")
      for batch in loop:
        src = batch['src'].to(device)
        tgt = batch['tgt'].to(device)

        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]

        output = model(src, tgt_input, teacher_forcing_ratio=0.0)  # No teacher forcing during validation
        output = output.reshape(-1, output.shape[-1])
        tgt_output = tgt_output.reshape(-1)
        loss = criterion(output, tgt_output)

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())
        wandb.log({"val_batch_loss": loss.item()})
    return total_loss / len(dataloader)

# utils.py
def get_model(name, config):
    if name == 'lstm':
        return
    elif name == 'transformer':
        return Seq2SeqTransformer(**config)
    else:
        raise ValueError(f"Unknown model name: {name}")


Cài đặt mô hình Transformers

In [ ]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2) * -(math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return x

In [ ]:
from torch.cuda.amp import autocast

In [ ]:
class Seq2SeqTransformer(BaseSeq2Seq):
    def __init__(self, input_dim, output_dim, embed_dim, num_heads, hidden_dim, num_layers, src_pad_idx, tgt_pad_idx, device):
        super().__init__()
        self.device = device
        self.src_tok_emb = nn.Embedding(input_dim, embed_dim, padding_idx=src_pad_idx)
        self.tgt_tok_emb = nn.Embedding(output_dim, embed_dim, padding_idx=tgt_pad_idx)
        self.positional_encoding = PositionalEncoding(embed_dim)

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout= 0.3)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout= 0.3)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        self.fc_out = nn.Linear(embed_dim, output_dim)
        self.src_pad_idx = src_pad_idx
        self.tgt_pad_idx = tgt_pad_idx

    def make_tgt_mask(self, tgt):
        tgt_len = tgt.size(1)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt_len).to(self.device)
        return tgt_mask

    def forward(self, src, tgt, teacher_forcing_ratio=1.0):
        src_mask = None
        tgt_mask = self.make_tgt_mask(tgt)
        src_padding_mask = (src == self.src_pad_idx)
        tgt_padding_mask = (tgt == self.tgt_pad_idx)

        src = self.positional_encoding(self.src_tok_emb(src))
        tgt = self.positional_encoding(self.tgt_tok_emb(tgt))

        memory = self.encoder(src.transpose(0, 1), src_key_padding_mask=src_padding_mask)
        output = self.decoder(
            tgt.transpose(0, 1),
            memory,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=src_padding_mask
        )

        output = self.fc_out(output.transpose(0, 1))  # [B, T, V]
        return output

    # greedy encoding
    def generate(self, src, max_len=50, beam_size=1):
      self.eval()
      with torch.no_grad(), autocast():
          batch_size = src.size(0)
          device = src.device
          sos_id = target_tokenizer.token_to_id("<sos>")
          eos_id = target_tokenizer.token_to_id("<eos>")

          self.to(device)
          src_mask = None
          src_padding_mask = (src == self.src_pad_idx).to(device)
          src_embed = self.positional_encoding(self.src_tok_emb(src))
          memory = self.encoder(src_embed.transpose(0, 1), src_key_padding_mask=src_padding_mask)
          memory = memory.transpose(0, 1)

          sequences = torch.full((batch_size, 1), sos_id, dtype=torch.long, device=device)
          ended = torch.zeros(batch_size, dtype=torch.bool, device=device)

          for _ in range(max_len):
              tgt_mask = self.make_tgt_mask(sequences).to(device)
              tgt_embed = self.positional_encoding(self.tgt_tok_emb(sequences))
              out = self.decoder(
                  tgt_embed.transpose(0, 1),
                  memory.transpose(0, 1),
                  tgt_mask=tgt_mask,
                  memory_key_padding_mask=src_padding_mask
              )
              out = self.fc_out(out.transpose(0, 1))[:, -1, :]
              probs = torch.softmax(out, dim=-1)
              next_tokens = probs.argmax(dim=-1)
              sequences = torch.cat([sequences, next_tokens.unsqueeze(1)], dim=1)
              ended = ended | (next_tokens == eos_id)
              if ended.all():
                  break

          return sequences


In [ ]:
import torch
wandb.init(
    project= "nmt-transformer",
    name= "nmt-transformer-run-4"
)
config = {
    "input_dim": len(vocab_src),         # số token trong vocab nguồn
    "output_dim": len(vocab_tgt),        # số token trong vocab đích
    "embed_dim": 256,
    "hidden_dim": 512,
    "num_layers": 6,
    "num_heads": 8,
    "src_pad_idx": vocab_src["<pad>"],
    "tgt_pad_idx": vocab_tgt["<pad>"],
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}
num_epochs = 16


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 23020350duong (23020350duong-vietnam-national-university-hanoi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
model = get_model("transformer", config).to(config["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=config["tgt_pad_idx"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, verbose=True)
patience = 3
counter = 0
best_val_loss = float('inf')
for epoch in range(num_epochs):
    print('Epoch:',epoch+1)
    train_loss = train_epoch(model, dataloader, optimizer, criterion, config["device"])
    wandb.log({"Epoch": epoch+1, "train_loss": train_loss})

    val_loss = validate_epoch(model, valid_data, criterion, "cuda")
    wandb.log({"epoch": epoch+1, "val_loss": val_loss, "learning_rate": optimizer.param_groups[0]['lr']})

    scheduler.step(val_loss)
    if val_loss < best_val_loss:
      best_val_loss = val_loss
      counter = 0
      torch.save(model.state_dict(), 'best_model_2.pt')
    else:
      counter += 1
      if counter >= patience:
        print('early stopping')
        break
wandb.finish()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch: 1
>> Start training


training:   0%|          | 0/1563 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
validating: 100%|██████████| 157/157 [00:07<00:00, 20.07it/s, loss=4.76]


Epoch: 2
>> Start training


validating: 100%|██████████| 157/157 [00:07<00:00, 19.89it/s, loss=4.63]


...

Epoch: 15
>> Start training


validating: 100%|██████████| 157/157 [00:07<00:00, 19.94it/s, loss=3.29]


Epoch: 16
>> Start training


validating: 100%|██████████| 157/157 [00:07<00:00, 19.78it/s, loss=2.85]


Epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
batch_loss,██▇▆▅▆▅▅▅▅▄▄▄▄▄▃▃▃▂▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁
epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▆▅▄▃▃▃▂▂▂▂▂▁▁▁▁
val_batch_loss,▇██▆▇▄▄▅▆▅▄▄▃▃▂▄▅▄▂▃▂▂▃▂▂▂▂▃▁▃▂▁▂▃▁▂▂▂▂▄
val_loss,█▆▅▄▄▃▃▃▂▂▂▂▁▁▁▁
Epoch,16
batch_loss,2.62288
epoch,16
learning_rate,0.0001


test nhỏ lẻ

In [ ]:
model.load_state_dict(torch.load("best_model_2.pt", map_location='cuda'))
model.eval()

Seq2SeqTransformer(
  (src_tok_emb): Embedding(16000, 256, padding_idx=0)
  (tgt_tok_emb): Embedding(16000, 256, padding_idx=0)
  (positional_encoding): PositionalEncoding()
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=512, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
        (linear2): Linear(in_features=512, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.3, inplace=False)
        (dropout2): Dropout(p=0.3, inplace=False)
      )
    )
  )
  (decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerDecoderLayer(
        (self_attn):

In [ ]:
sentence = "we are playing on the ground"
encode_sentence = source_tokenizer.encode(sentence)
id = encode_sentence.ids
src_id = [source_tokenizer.token_to_id('<sos>')] + id + [source_tokenizer.token_to_id('<eos>')]
# Đưa vào tensor
src_tensor = torch.tensor([src_id], dtype=torch.long).to('cuda')

In [ ]:
output_ids = model.generate(src_tensor, max_len=100)
output = target_tokenizer.decode(output_ids.squeeze().tolist())
output

/tmp/ipython-input-26-905772382.py:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():


' chúng ta đang chơi trên mặt đất.'

In [ ]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.3 MB/s eta 0:00:00


In [ ]:
from sacrebleu import corpus_bleu

def compute_bleu(model, dataloader, device, tokenizer, config, num_examples_to_print=5):
    model.eval()
    hypotheses = []
    references = []

    config["tgt_pad_idx"] = target_tokenizer.token_to_id("<pad>")
    config["tgt_sos_idx"] = target_tokenizer.token_to_id("<sos>")
    config["tgt_eos_idx"] = target_tokenizer.token_to_id("<eos>")

    special_tokens = {
        config["tgt_pad_idx"],
        config["tgt_sos_idx"],
        config["tgt_eos_idx"]
    }

    with torch.no_grad():
        for batch in dataloader:
            src = batch['src'].to(device)
            tgt = batch['tgt'].to(device)

            output_ids = model.generate(src, max_len=100)

            for ref_ids, hyp_ids in zip(tgt.tolist(), output_ids.tolist()):
                # Loại bỏ token đặc biệt
                ref_tokens = [t for t in ref_ids if t not in special_tokens]
                hyp_tokens = [t for t in hyp_ids if t not in special_tokens]

                ref_text = tokenizer.decode(ref_tokens)
                hyp_text = tokenizer.decode(hyp_tokens)

                references.append(ref_text)
                hypotheses.append(hyp_text)

    # BLEU yêu cầu references là list của list
    bleu = corpus_bleu(hypotheses, [references])

    print(f"\n===> BLEU score: {bleu.score:.2f}")

    # Hiển thị một số cặp câu để kiểm tra
    print("\n===> Sample translations:")
    for i in range(min(num_examples_to_print, len(hypotheses))):
        print(f"REF: {references[i]}")
        print(f"HYP: {hypotheses[i]}")
        print("-" * 50)

    return bleu.score

In [ ]:
score = compute_bleu(model, valid_data, "cuda", target_tokenizer, config= config, num_examples_to_print=5)
score

/tmp/ipython-input-26-905772382.py:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():



===> BLEU score: 5.04

===> Sample translations:
REF:  ông không bao giờ yêu con, johnny.
HYP:  ông không bao giờ yêu cầu bạn,..................... anh...... anh. anh. anh không.. anh. anh không bao giờ. anh.
--------------------------------------------------
REF:  tôi xin lỗi vince, tôi khá bận.
HYP:  tôi xin lỗi về điều đó, v............................. tôi...........
--------------------------------------------------
REF:  tìm được mấy vỏ đạn này
HYP:  và chúng tôi tìm thấy những người này............................... và chúng tôi tìm thấy và chúng tôi tìm thấy....
--------------------------------------------------
REF:  và những chủ quyền, có tính chất xuyên xuốt, liên kết với nhau một cách toàn cầu này, theo một vài cách nào đó thách thức chủ quyền của các quốc gia theo những cách rất thú vị nhưng đôi khi cũng hành động vượt ra và mở rộng nó vào khi mà điều khiển vượt quá những gì mọi người có thể hoặc không thể làm với thông tin có nhiều tác động hơn bao giờ hết đối với việc 

5.041041163822801